In [1]:
import jieba
import warnings
warnings.filterwarnings('ignore')

#### 中文分词

支持三种分词模式： <br>
精确模式，试图将句子最精确地切开，适合文本分析； <br>
全模式，把句子中所有的可以成词的词语都扫描处理，速度非常快，但是不能解决歧义； <br>
搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于引擎分词

In [5]:
"""
jieba.cut(sentence=, cut_all=, HMM=)
sentence: 分词语句
cut_all: True/False, 是否采用全模式， 默认False
HMM: 是否采用HMM模型

RETURN: 生成器
"""
sent1 = "从头开始梳理NLP的知识体系"
'/'.join(jieba.cut(sent1))

'从头开始/梳理/NLP/的/知识/体系'

In [6]:
# cut全模式分词
'/'.join(jieba.cut(sent1, cut_all=True))

'从头/从头开始/开始/梳理/NLP/的/知识/识体/体系'

In [7]:
"""
cut_for_search  搜索引擎模式 分词
参数同cut
"""
'/'.join(jieba.cut_for_search(sent1))

'从头/开始/从头开始/梳理/NLP/的/知识/体系'

In [8]:
"""
lcut和lcut_for_search分别作为cut和cut_for_search的衍生，直接获取列表对象
"""
print(jieba.lcut(sent1))
print(jieba.lcut_for_search(sent1))

['从头开始', '梳理', 'NLP', '的', '知识', '体系']
['从头', '开始', '从头开始', '梳理', 'NLP', '的', '知识', '体系']


#### 添加自定义词或词典

通过添加自定义词或词典，以便对jieba词库里没有的词进行切分 <br>
jieba.add_word: 添加词<br>jieba.del_word: 删除词<br>jieba.initialize: 初始化字典<br> jieba.set_dictionary: 设置词典<br>  jieba.load_userdict: 添加词典<br>  

In [10]:
# 对于专业或非常见词，需自定义词
'/'.join(jieba.cut("通过活检和免疫组化，基因分析等多项检查，医生诊断为滤泡淋巴瘤"))

'通过/活检/和/免疫组化/，/基因/分析/等/多项/检查/，/医生/诊断/为/滤泡/淋巴瘤'

In [12]:
"""
jieba.add_word(word=, freq=, tag=)
word: 需要添加的词
freq: 词频
tag: 词性
freq和tag非必填
"""
jieba.add_word('基因分析')
'/'.join(jieba.cut("通过活检和免疫组化，基因分析等多项检查，医生诊断为滤泡淋巴瘤"))

'通过/活检/和/免疫组化/，/基因分析/等/多项/检查/，/医生/诊断/为/滤泡/淋巴瘤'

In [17]:
"""
jieba.load_dictionary(f)
f: 字典地址或文件指针, 字典文档格式每行为  word \s frequency \s tag,  frquency和tag非必须
"""
jieba.initialize()
jieba.load_userdict('./user_dict.txt')
'/'.join(jieba.cut("通过活检和免疫组化，基因分析等多项检查，医生诊断为滤泡淋巴瘤"))

'通过/活检/和/免疫组化/，/基因分析/等/多项/检查/，/医生/诊断/为/滤泡淋巴瘤'

#### 更改词频

jieba的内部算法中进行DAG最大概率搜索时，会统计词频<br>
jieba.suggest_freq(segment, tune=False)

case1: 强行拆分

In [37]:
# 中将的频率
print("中将: {} , 中: {}".format(jieba.get_FREQ("中将"), jieba.get_FREQ("中")))

中将: 763 , 中: 243191


In [39]:
jieba.lcut("如果放到post中将出错", HMM=False)

['如果', '放到', 'post', '中将', '出错']

In [40]:
jieba.suggest_freq(("中", "将"), True)   # tune设为True使其生效
jieba.lcut("如果放到post中将出错", HMM=False)    # 注意HMM需设为False，否则会影响分词结果

['如果', '放到', 'post', '中', '将', '出错']

case2：强行组合

In [41]:
jieba.lcut("「台中」发生了龙卷风")

['「', '台', '中', '」', '发生', '了', '龙卷风']

In [42]:
jieba.suggest_freq("台中", True)   # tune设为True使其生效, S
jieba.lcut("「台中」发生了龙卷风", HMM=False)

['「', '台中', '」', '发生', '了', '龙卷风']

case3: 识别【带空格的词】

In [2]:
import re
sentence = 'Blade Master疾风刺杀Archmage'
jieba.add_word('Blade Master')  # 添词
print('修改前：', jieba.lcut(sentence))
jieba.re_han_default = re.compile('(.+)', re.U)  # 修改格式
print('修改后：', jieba.lcut(sentence))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/rk/nnl9yhm55kb6325ckffkn6hw0000gn/T/jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built succesfully.


修改前： ['Blade', ' ', 'Master', '疾风', '刺杀', 'Archmage']
修改后： ['Blade Master', '疾风', '刺杀', 'Archmage']


#### 停用词

jieba中没有直接的停用词接口，需手动进行加载

In [47]:
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

In [50]:
def sentence_seg(sentence, stopwords):
    seg = []
    for word in jieba.cut(sentence=sentence):
        if word not in stopwords:
            seg.append(word)
    return seg

In [45]:
# 增加停用词前
jieba.lcut("今天下了一场大雨，好多人都被淋湿了")

['今天', '下', '了', '一场', '大雨', '，', '好多', '人', '都', '被', '淋湿', '了']

In [51]:
# 增加停用词后
sentence_seg("今天下了一场大雨，好多人都被淋湿了", stopwordslist('../StopWords/stop_words_1.txt'))

['今天', '一场', '大雨', '好多', '淋湿']

#### 分词同时定位

In [55]:
"""
jieba.tokenize(unicode_sentence, mode="default", HMM=True)
mode: "default" 或 "search"（类似于cut_for_search）
其内部调用了jieba.cut命令
"""
for item in jieba.tokenize("今天天气不错"):
    print("word:{}, start:{}, end:{}".format(item[0], item[1], item[2]))
print("-"*30)
for item in jieba.tokenize("今天天气不错",mode="search"):
    print("word:{}, start:{}, end:{}".format(item[0], item[1], item[2]))

word:今天天气, start:0, end:4
word:不错, start:4, end:6
------------------------------
word:今天, start:0, end:2
word:天天, start:1, end:3
word:天气, start:2, end:4
word:今天天气, start:0, end:4
word:不错, start:4, end:6


#### 分词器

In [57]:
"""
tokenizer = jieba.Tokenizer(dictionary=)
通过不同的字典定义不同的分词器进行分词
"""
tokenizer = jieba.Tokenizer()
tokenizer.lcut("今天天气不错")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/rk/nnl9yhm55kb6325ckffkn6hw0000gn/T/jieba.cache
Loading model cost 0.790 seconds.
Prefix dict has been built succesfully.


['今天天气', '不错']

#### 并行分词

In [58]:
import time

with open('./data/cutdata', 'r', encoding='utf-8') as f:
    sent2 = f.read()
print(sent2)

沙瑞金赞叹易学习的胸怀，是金山的百姓有福，可是这件事对李达康的触动很大。易学习又回忆起他们三人分开的前一晚，大家一起喝酒话别，易学习被降职到道口县当县长，王大路下海经商，李达康连连赔礼道歉，觉得对不起大家，他最对不起的是王大路，就和易学习一起给王大路凑了5万块钱，王大路自己东挪西撮了5万块，开始下海经商。没想到后来王大路竟然做得风生水起。沙瑞金觉得他们三人，在困难时期还能以沫相助，很不容易。



In [59]:
# 不并行分词
t1 = time.time()
jieba.cut(sent2)
t2 = time.time()
print(t2-t1)

6.318092346191406e-05


In [60]:
# 并行分词
jieba.enable_parallel(4)
t1 = time.time()
jieba.cut(sent2)
t2 = time.time()
print(t2-t1)

0.0001399517059326172
